# Deploy an MLflow model with SageMaker

## Install MLflow

In [1]:
!pip install -q mlflow==2.0.1

In [2]:
!pip install scikit-learn==1.1.3

## Setup environment

In [15]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import load_boston
from mlflow.deployments import get_deploy_client

# name of the AWS region to which to deploy the application
region = sagemaker.Session().boto_region_name
# we are using the notebook instance role for training in this example
role = sagemaker.get_execution_role() 
# uri of your remote mlflow server
tracking_uri = 'http://MLflow-MLFLO-w9tpeQwzgqZz-45551a4ba3a1ef8d.elb.us-east-1.amazonaws.com/' 
# set remote mlflow server
mlflow.set_tracking_uri(tracking_uri)

## Build MLflow docker image to serve the model with SageMaker 

### Instructions to Build Your Own MLflow Docker Image to Serve a Model

You can do this on your local Docker daemon.

1. Make sure you have configured AWS credentials on your PC:

    ```sh
    $ aws configure
    AWS Access Key ID [****]: Your-Access-Key-ID
    AWS Secret Access Key [****]: Your-Secret-Access-Key
    Default region name [None]: us-east-1
    Default output format [None]: json
    ```

2. Install Docker Desktop https://www.docker.com/products/docker-desktop/ (ensure your Docker Desktop is up and running before proceeding with the following steps).

3. Install MLflow:

    ```sh
    $ pip install mlflow
    ```

4. Install Boto3:

    ```sh
    $ pip install boto3
    ```

5. Run the following command to build and push the MLflow container. This step will take 20 to 30 minutes to complete and will automatically push the container to Amazon ECR:

    ```sh
    $ mlflow sagemaker build-and-push-container
    ```

Once you have your own Docker image, you can start using it.


In [ ]:
#!aws ecr get-login --no-include-email --registry-ids 146171926552 --region us-east-1

In [41]:
##MLFlow Docker Setup

In [60]:
!docker images

REPOSITORY                                            TAG       IMAGE ID       CREATED             SIZE
03sarath/mlflow-pyfunc                                2.12.2    bef1a9579b21   About an hour ago   2.49GB
<none>                                                <none>    aacb37258e6a   3 hours ago         124MB
ubuntu                                                20.04     2abc4dfd8318   2 weeks ago         72.8MB
146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.0.1     7c5a6df04b5b   2 months ago        2.8GB
146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.12.1    7c5a6df04b5b   2 months ago        2.8GB
146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.12.2    7c5a6df04b5b   2 months ago        2.8GB
03sarath/mlflow-pyfunc                                2.0.1     7c5a6df04b5b   2 months ago        2.8GB
public.ecr.aws/d2p5i2f8/mlflow/mlflow-pyfunc          2.0.1     7c5a6df04b5b   2 months ago        2.8GB


In [43]:
!docker pull 03sarath/mlflow-pyfunc:2.12.2 #Pull Docker images from my registry

2.12.2: Pulling from 03sarath/mlflow-pyfunc

c94e5e10: Already exists 
1a841b62: Pulling fs layer 
92926c51: Pulling fs layer 
4141f19b: Pulling fs layer 
d50a325e: Pulling fs layer 
2a40ba8d: Pulling fs layer 
e46d18db: Pulling fs layer 
75548dfe: Pulling fs layer 
b23d2876: Pulling fs layer 
755c6ff6: Pulling fs layer 
9c3cf88b: Pulling fs layer 
22d7abdc: Pulling fs layer 
6493764b: Pulling fs layer 
6e1a3d00: Pulling fs layer 
8e2e887d: Pulling fs layer 
Digest: sha256:e14fd53a9ff37d91cbf775c02749bdc16cff931e482cfbabd4e0f904f85ce41fExtracting  127.6MB/164.1MB
Status: Downloaded newer image for 03sarath/mlflow-pyfunc:2.12.2
docker.io/03sarath/mlflow-pyfunc:2.12.2


In [44]:
!docker images

REPOSITORY                                            TAG       IMAGE ID       CREATED          SIZE
03sarath/mlflow-pyfunc                                2.12.2    bef1a9579b21   42 minutes ago   2.49GB
<none>                                                <none>    aacb37258e6a   2 hours ago      124MB
ubuntu                                                20.04     2abc4dfd8318   2 weeks ago      72.8MB
public.ecr.aws/d2p5i2f8/mlflow/mlflow-pyfunc          2.0.1     7c5a6df04b5b   2 months ago     2.8GB
146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.0.1     7c5a6df04b5b   2 months ago     2.8GB
03sarath/mlflow-pyfunc                                2.0.1     7c5a6df04b5b   2 months ago     2.8GB


In [49]:
#!docker tag 03sarath/mlflow-pyfunc:2.12.2 <You'r ECR repo ID>
!docker tag 03sarath/mlflow-pyfunc:2.12.2 146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2

In [50]:
!docker images

REPOSITORY                                            TAG       IMAGE ID       CREATED          SIZE
03sarath/mlflow-pyfunc                                2.12.2    bef1a9579b21   46 minutes ago   2.49GB
<none>                                                <none>    aacb37258e6a   2 hours ago      124MB
ubuntu                                                20.04     2abc4dfd8318   2 weeks ago      72.8MB
03sarath/mlflow-pyfunc                                2.0.1     7c5a6df04b5b   2 months ago     2.8GB
public.ecr.aws/d2p5i2f8/mlflow/mlflow-pyfunc          2.0.1     7c5a6df04b5b   2 months ago     2.8GB
146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.0.1     7c5a6df04b5b   2 months ago     2.8GB
146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.12.1    7c5a6df04b5b   2 months ago     2.8GB
146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow   2.12.2    7c5a6df04b5b   2 months ago     2.8GB


In [51]:

#!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin <URI OF YOUR REPO>
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [52]:
#!docker push <You'r ECR repo ID>
!docker push 146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2

The push refers to repository [146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow]

e7110687: Preparing 
2ac6be2c: Preparing 
af080466: Preparing 
fca9b552: Preparing 
ff424f2c: Preparing 
36c98e78: Preparing 
28051161: Preparing 
6be9fffc: Preparing 
6ff92d60: Preparing 
e7c76678: Preparing 
54a07c6d: Preparing 
8b0211f3: Preparing 
528d1c04: Preparing 
a3817058: Preparing 
05a870f4: Preparing 
bef40418: Preparing 
f2c85376: Preparing 
8cc4052c: Preparing 
0715cd5f: Preparing 
0445c449: Layer already exists 2.12.2: digest: sha256:72cdb5089a74f3fd8e59e873368c3f674ca20c32c2c0c9de94b904a57d3b4e1c size: 4529


In [53]:
# URL of the ECR-hosted Docker image the model should be deployed into
image_uri = '146171926552.dkr.ecr.us-east-1.amazonaws.com/mlflow:2.12.2'

## Deploy a SageMaker endpoint with our scikit-learn model

In [54]:
endpoint_name = 'boston-housing-mlops'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 'models:/boston/1'

In [55]:
config={
    'execution_role_arn': role,
    'image_url': image_uri,
    'instance_type': 'ml.m5.xlarge',
    'instance_count': 1, 
    'region_name': region
}

client = get_deploy_client("sagemaker")

client.create_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    flavor='python_function',
    config=config
)

2024/05/17 12:50:05 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2024/05/17 12:50:05 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
2024/05/17 12:50:06 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-east-1-146171926552` already exists. Skipping creation.
2024/05/17 12:50:06 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': '1ZV2221SZXWMGDSP', 'HostId': 'k4ju2OudV3VaONNMZqR1rdujAWQrZDCtRs8pnBogFpjyyYoXk385xRwpIF51cuz0pX/0SUC/lY4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'k4ju2OudV3VaONNMZqR1rdujAWQrZDCtRs8pnBogFpjyyYoXk385xRwpIF51cuz0pX/0SUC/lY4=', 'x-amz-request-id': '1ZV2221SZXWMGDSP', 'date': 'Fri, 17 May 2024 12:50:07 GMT', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}}
2024/05/17 12:50:06 INFO mlflow.sagemaker: Creating new endpoint with name: boston-housing-mlops-jan ...
2024/05/17 12:50:07 INFO mlflow.sagemaker: Created model with arn: arn:aws:sagemaker:

{'name': 'boston-housing-mlops-jan', 'flavor': 'python_function'}

## Predict

In [56]:
# load boston dataset
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [57]:
client = get_deploy_client(f"sagemaker:/{region}")

payload = df.iloc[[0]]
prediction = client.predict(endpoint_name, df.iloc[[3]])

print(f'Payload: {payload}')
print(f'Prediction: {prediction}')

Payload:       CRIM    ZN  INDUS  CHAS    NOX     RM   AGE   DIS  RAD    TAX  PTRATIO  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.09  1.0  296.0     15.3   

       B  LSTAT  
0  396.9   4.98  
Prediction: {'predictions': [35.16046144133643]}


## Delete endpoint

In [59]:
client.delete_deployment('boston-housing-mlops', config=config)

2024/05/17 13:06:31 INFO mlflow.sagemaker: Deleted endpoint with arn: arn:aws:sagemaker:us-east-1:146171926552:endpoint/boston-housing-mlops-jan
2024/05/17 13:06:31 INFO mlflow.sagemaker: Waiting for the delete operation to complete...
2024/05/17 13:06:31 INFO mlflow.sagemaker: Deletion is still in progress. Current endpoint status: Deleting
2024/05/17 13:06:36 INFO mlflow.sagemaker: The deletion operation completed successfully with message: "The SageMaker endpoint was deleted successfully."
2024/05/17 13:06:36 INFO mlflow.sagemaker: Cleaning up unused resources...
2024/05/17 13:06:36 INFO mlflow.sagemaker: Deleted associated endpoint configuration with arn: arn:aws:sagemaker:us-east-1:146171926552:endpoint-config/boston-housing-mlops-jan-config--hk00up6tqulfav0gpg9abg
2024/05/17 13:06:37 INFO mlflow.sagemaker: Deleted associated model with arn: arn:aws:sagemaker:us-east-1:146171926552:model/boston-housing-mlops-jan-model-jdgirltzrqyqftxr0kb4lg
